In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# expandable_segments allows the allocator to create a segment initially and then expand its size later when more memory is needed.
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [3]:
import tracemalloc
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt

import torch

from src.datasets import DatasetBuilder
from src.utils import seed_everything, get_config, load_model_weights, evaluate_classification_model
from src.cf_methods.coin import CounterfactualCGAN, CounterfactualTrainer

seed_everything()

mkdir -p failed for path /.cache/matplotlib: [Errno 13] Permission denied: '/.cache'
Matplotlib created a temporary cache directory at /tmp/matplotlib-97987kua because there was an issue with the default path (/.cache/matplotlib); it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Matplotlib is building the font cache; this may take a moment.
2025-05-16 16:24:28.736568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747405468.754842 1310242 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747405468.760310 1310242 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cu

In [4]:
# Prepare cuda to make a snapshot of the allocated memory  
# torch.cuda.memory._record_memory_history()

In [4]:
config_dir = '/data/leuven/365/vsc36567/CF-Robustness-Benchmark/configs' #'D:\PycharmProjects\CF-Robustness-Benchmark\configs' #
config_path = osp.join(config_dir, 'coin_derma.yaml') 
config = get_config(config_path) 

### Load the dataset

In [5]:
ds_builder = DatasetBuilder(config)
ds_builder.setup()
train_loader, val_loader, test_loader = ds_builder.get_dataloaders()

### Build the model and trainer

For the training of the COIN that uses a CF-CGAN, we need the model itself and the corresponding trainer 

In [6]:
cfcgan = CounterfactualCGAN(opt=config, img_size=config.data.img_size)

Using SNConv in generator: False
Generator in channels [1024, 1536, 768, 384, 192]
Generator out channels [1024, 512, 256, 128, 64]
Using SNConv in generator: False
Using perturbation fuse scheme: skip_add_tanh


In [7]:
continue_path = '/data/leuven/365/vsc36567/CF-Robustness-Benchmark/cf_output/derma/coin_cfe-May-15-2025_05+53PM-exp'
trainer = CounterfactualTrainer(opt=config, model=cfcgan, continue_path=continue_path)

[2025-05-16 14:31:18|INFO] - ================ Session (Fri May 16 14:31:18 2025) ================
[2025-05-16 14:31:18|INFO] - Logging directory: /data/leuven/365/vsc36567/CF-Robustness-Benchmark/cf_output/derma/coin_cfe-May-15-2025_05+53PM-exp
[2025-05-16 14:31:39|INFO] - Restored checkpoint /data/leuven/365/vsc36567/CF-Robustness-Benchmark/cf_output/derma/coin_cfe-May-15-2025_05+53PM-exp/checkpoints/checkpoint_2.pth (May 15, 2025)


### Training

In [8]:
trainer.fit([train_loader, val_loader]) 

In [12]:
# Finilizing a snapshot of the memory allocation
# torch.cuda.memory._dump_snapshot("my_snapshot.pickle")
# torch.cuda.memory._record_memory_history(enabled=None)

### Estimation of allocated memory

In [7]:
def get_model_memory_usage(model: torch.nn.Module):
    total_params = sum(p.numel() for p in model.parameters())
    param_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
    buffer_size_bytes = sum(b.numel() * b.element_size() for b in model.buffers())
    total_size_bytes = param_size_bytes + buffer_size_bytes
    total_size_mb = total_size_bytes / (1024 ** 2)

    print(f"Total parameters: {total_params}")
    print(f"Total size (parameters + buffers): {total_size_mb:.2f} MB")

# Example usage
get_model_memory_usage(cfcgan)


Total parameters: 96166150
Total size (parameters + buffers): 367.14 MB


In [8]:
print(f"Allocated: {torch.cuda.memory_allocated() / (1024**2):.2f} MB")
print(f"Reserved:  {torch.cuda.memory_reserved() / (1024**2):.2f} MB")

torch.cuda.reset_peak_memory_stats()

Allocated: 277.27 MB
Reserved:  384.00 MB


In [9]:
# Calculation of approximate memory required for one batch
input = torch.randn(config.batch_size, 3, 224, 224)  # shape = (16, 3, 224, 224)
input_bytes = input.numel() * input.element_size()
input_MB = input_bytes / (1024**2)
print(f"Input batch takes ~{input_MB:.2f} MB")

Input batch takes ~9.19 MB
